In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
tf.__version__

'2.11.0'

In [3]:
dataset,info = tfds.load('imdb_reviews/subwords8k',
                         with_info=True,
                         as_supervised=True)

2023-04-10 10:23:05.239461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 10:23:06.022545: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 10:23:06.022662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 10:23:06.030531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-10 10:23:06.030863: I tensorflow/compile

In [4]:
tD,vD = dataset['train'],dataset['test']

In [5]:
encoder = info.features['text'].encoder #used for encoding

In [6]:
s = 'Hello, how are you'
es = encoder.encode(s)

In [7]:
es #encoding string

[4025, 8040, 2, 113, 29, 151]

In [8]:
encoder.vocab_size #total words in vocab

8185

In [9]:
for idx in es:
    print('{}-->{}'.format(idx,encoder.decode([idx])))

4025-->Hell
8040-->o
2-->, 
113-->how 
29-->are 
151-->you


In [10]:
tD = tD.shuffle(1000) #randomly pick 1000 training data
tD = tD.padded_batch(50) #create batches of 50
vD = vD.padded_batch(50) #create batches of 50

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size,10), #Feature extraction through encoding 
    tf.keras.layers.LSTM(12), #use LSTM in RNN, 12 units in LSTM
    tf.keras.layers.Dense(5), #hidden layer with 5 neurons
    tf.keras.layers.Dense(1), #1 unit or neuron , softmax layer as binary classification problem
])

In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= tf.keras.optimizers.Adam(1e-3),
              metrics= ['accuracy'])

In [13]:
history = model.fit(tD,epochs=2,
                    validation_data=vD,
                    validation_steps=30)

Epoch 1/2


2023-04-10 10:23:39.803630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-04-10 10:23:42.141006: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fbbfb472930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-10 10:23:42.141107: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2023-04-10 10:23:42.181242: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-10 10:23:42.847963: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 [==============================] - 157s 270ms/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.4993
Epoch 2/2
  1/500 [..............................] - ETA: 1:21 - loss: 0.6932 - accuracy: 0.5000

2023-04-10 10:25:59.947790: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


500/500 [==============================] - 105s 211ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6935 - val_accuracy: 0.4993


2023-04-10 10:27:45.453187: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
model.save("TCmodel.h5")

In [15]:
reconstructed_model = tf.keras.models.load_model("TCmodel.h5")

In [16]:
vL,vA = model.evaluate(vD)
vL,vA = reconstructed_model.evaluate(vD)

500/500 [==============================] - 29s 56ms/step - loss: 0.6932 - accuracy: 0.5004


In [15]:
print(vL,vA)

0.6930391192436218 0.5
